In [ ]:
! pip3 install requests 
! pip3 install beautifulsoup4 

import requests as rq

from bs4 import BeautifulSoup
import numpy as np

In [ ]:
urls = 'https://www.lamoda.ru/c/2512/clothes-muzhskie-futbolki/#breadcrumbs','https://www.lamoda.ru/c/2512/clothes-muzhskie-futbolki/?page=2','https://www.lamoda.ru/c/2512/clothes-muzhskie-futbolki/?page=3', 'https://www.lamoda.ru/c/2512/clothes-muzhskie-futbolki/?page=4','https://www.lamoda.ru/c/2512/clothes-muzhskie-futbolki/?page=5','https://www.lamoda.ru/c/2512/clothes-muzhskie-futbolki/?page=6','https://www.lamoda.ru/c/2512/clothes-muzhskie-futbolki/?page=7'

counter = 0
for url in urls:
    r = rq.get(url)
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    souplist = soup.find_all('a', {'href' : True})
    for tag in souplist:
        if 'futbolka' in tag["href"]:
            counter += 1
            url = '/'.join(url.split('/')[:3])
            child_url = f'{url}{tag["href"]}'
            #         print(child_url)
            r = rq.get(child_url)
            html = r.text
            with open(f'{counter}.html', 'w', encoding='utf-8') as fw:
                fw.write(html)

In [3]:
import os
import re

In [45]:
data = []
for filename in os.listdir():
    if filename.endswith('html'):
        with open(filename, encoding='utf-8') as f:
            html = f.read()
            ld_soup = BeautifulSoup(html, 'html.parser')#открываем каждую страницу в папке
            #print(ld_soup)
            reviews = ld_soup.select('[data-rating]')#делаем из каждой страницы суп
           #print(reviews)
            for review in reviews:
                if not review['data-rating']:
                    print(review)
                    break
                else:
                    data.append((review['data-rating'], 
                                 review.find('span', class_='rev__node-head_author').get_text(),
                                 review.find('div', class_='rev__node-text').get_text()))#достаем с каждой страницы текст ревью и складываем их в кортеж

In [46]:
print(len(data))

1644


In [5]:
! pip3 install pandas
import pandas as pd

You should consider upgrading via the 'C:\Users\ДНС\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [47]:
reviews_m = df = pd.DataFrame(data, columns=['stars', 'name', 'feedback'])
reviews_m

,stars,name,feedback
0,100,Павел,"\n Материал плотный, хороший! Очень сти..."
1,100,Тарабрин Николай,\n Крутая майка. Материал плотный качет...
2,100,Надежда,"\n отлично выглядит, отличное качество ..."
3,100,Горбатов Николай,\n Гуд\n\n\n
4,100,Горбатов Николай,\n Гуд\n\n\n
...,...,...,...
1639,100,Кирилл,"\n Принт классный, да и в принципе хоро..."
1640,100,Александра,"\n Качество крутое) и выглядит шикарно,..."
1641,100,регина,\n Все понравилось!!\n
1642,100,Shamil,\n Отличная футболка. 2pac forever !!!\...


In [8]:
stars = reviews_m['stars'].tolist()
stars_int = [(int(elem)) for elem in stars]
reviews_m = reviews_m.assign(stars=stars_int)#меняем буквенные оценки на числовые

In [9]:
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
nltk.download('stopwords')
from nltk import tokenize
import itertools
from pymorphy2 import MorphAnalyzer
from collections import Counter, defaultdict
import numpy as np
import string
from string import punctuation
from gensim.utils import tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ДНС\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
feedback = reviews_m['feedback'].tolist()
a = str(feedback)
print(len(a))

132310


In [11]:
feedback_m = reviews_m[reviews_m["name"].str.count('\w+[йбвгджзклмнпрстью]$')>0]
feedback_m#собираем только мужские отзывы регуляркой

,stars,name,feedback
0,100,Хейчиев Юрий,\n Отлично \n
1,100,Хейчиев Юрий,\n Отлично \n
2,100,Хейчиев Юрий,\n Отлично \n
3,100,дмитрий,\n По размеру и качеству все отлично\n ...
4,100,Ильнур,"\n Цвет совпадает, как на фото. Качеств..."
...,...,...,...
2092,100,Щеглов Владимир,\n Хорошая футболка\n
2093,100,Александр,"\n Качество хорошее, размер соответству..."
2096,100,алексей,\n ☝️🤟\n
2097,100,Павел,\n Хорошая футболка. \n


In [12]:
feedback_nik = reviews_m[reviews_m["name"].str.count('Никита')>0]
feedback_dan = reviews_m[reviews_m["name"].str.count('Данила')>0]#Подчищаем то, что не попало в регулярку 1

In [86]:
#feedback_dan

In [13]:
feedback_m_nik = feedback_m.append(feedback_nik)
feedback_m_dan = feedback_m_nik.append(feedback_dan)#Подчищаем то, что не попало в регулярку 2

In [87]:
#feedback_m_dan

In [14]:
feedback_m = feedback_m_dan[feedback_m_dan["name"].str.count('Аннет')!=1] #Подчищаем то, что не попало в регулярку 3
feedback_m = feedback_m[feedback_m["name"].str.count('Айгуль')!=1] # уточнить у Анастасии, правильно ли написано

In [15]:
feedback_m

,stars,name,feedback
0,100,Хейчиев Юрий,\n Отлично \n
1,100,Хейчиев Юрий,\n Отлично \n
2,100,Хейчиев Юрий,\n Отлично \n
3,100,дмитрий,\n По размеру и качеству все отлично\n ...
4,100,Ильнур,"\n Цвет совпадает, как на фото. Качеств..."
...,...,...,...
1488,100,Никита,"\n Хорошее качество, размер в размер 👌🏼..."
1633,100,Никита,"\n Не первая моя футболка данной фирмы,..."
1641,100,Никита,\n Отличная футболка\n
723,100,Данила,\n Хорошая футболка \n


In [16]:
feedback = feedback_m['feedback'].tolist()
feedback_tok = [' '.join(wordpunct_tokenize(elem)) for elem in feedback]#Убираем непечатные символы, но оставляем пунктуауию,
#чтобы посчитать среднюю длину предложения
#print(feedback_tok)

In [17]:
feedback_m = feedback_m.assign(feedback=feedback_tok)
feedback_m.head(10)

,stars,name,feedback
0,100,Хейчиев Юрий,Отлично
1,100,Хейчиев Юрий,Отлично
2,100,Хейчиев Юрий,Отлично
3,100,дмитрий,По размеру и качеству все отлично
4,100,Ильнур,"Цвет совпадает , как на фото . Качество хорошее ."
5,100,Михаил,За такую цену качество хорошее . Цвет понравил...
8,100,Любовь,Товар отличный ! Цвет особенно . Ничего лишнег...
10,100,Иван,Норм
15,80,матвиенко кирилл,Сойдёт
18,100,Александр,Класс


In [18]:
def averageSentence(b):
    a = sent_tokenize(b)
    num_list = []
    for elem in a:
        elem = elem.translate(str.maketrans('','',string.punctuation))
        num_list.append(len(elem.split()))
        sum(num_list)
        avg = sum(num_list)/len(a)
    return avg

In [19]:
feedback_m['avg_sent']= feedback_m['feedback'].apply(lambda x: averageSentence(x))


feedback_m.head(10)#посчитали среднюю длину предложения без знаков препинания

,stars,name,feedback,avg_sent
0,100,Хейчиев Юрий,Отлично,1.000000
1,100,Хейчиев Юрий,Отлично,1.000000
2,100,Хейчиев Юрий,Отлично,1.000000
3,100,дмитрий,По размеру и качеству все отлично,6.000000
4,100,Ильнур,"Цвет совпадает , как на фото . Качество хорошее .",3.500000
5,100,Михаил,За такую цену качество хорошее . Цвет понравил...,4.500000
8,100,Любовь,Товар отличный ! Цвет особенно . Ничего лишнег...,2.333333
10,100,Иван,Норм,1.000000
15,80,матвиенко кирилл,Сойдёт,1.000000
18,100,Александр,Класс,1.000000


In [20]:
feedback = feedback_m['feedback'].tolist()
d = str(feedback)
print(len(d))

45178


In [21]:
feedback_norm = [(elem.translate(str.maketrans('','',string.punctuation)))for elem in feedback_tok]

In [22]:
feedback_m = feedback_m.assign(feedback=feedback_norm)#Убрали знаки препинания, чтобы посчитать общую длину отзыва
feedback_m

,stars,name,feedback,avg_sent
0,100,Хейчиев Юрий,Отлично,1.0
1,100,Хейчиев Юрий,Отлично,1.0
2,100,Хейчиев Юрий,Отлично,1.0
3,100,дмитрий,По размеру и качеству все отлично,6.0
4,100,Ильнур,Цвет совпадает как на фото Качество хорошее,3.5
...,...,...,...,...
1488,100,Никита,Хорошее качество размер в размер 👌🏼,6.0
1633,100,Никита,Не первая моя футболка данной фирмы все как в...,11.0
1641,100,Никита,Отличная футболка,2.0
723,100,Данила,Хорошая футболка,2.0


In [23]:
words = feedback_m['feedback'].tolist()
num_words = [len(sentence.split()) for sentence in words]
feedback_m = feedback_m.assign(gender = 'M', num_words = num_words)#посчитали общую длину отзыва
feedback_m

,stars,name,feedback,avg_sent,gender,num_words
0,100,Хейчиев Юрий,Отлично,1.0,M,1
1,100,Хейчиев Юрий,Отлично,1.0,M,1
2,100,Хейчиев Юрий,Отлично,1.0,M,1
3,100,дмитрий,По размеру и качеству все отлично,6.0,M,6
4,100,Ильнур,Цвет совпадает как на фото Качество хорошее,3.5,M,7
...,...,...,...,...,...,...
1488,100,Никита,Хорошее качество размер в размер 👌🏼,6.0,M,6
1633,100,Никита,Не первая моя футболка данной фирмы все как в...,11.0,M,11
1641,100,Никита,Отличная футболка,2.0,M,2
723,100,Данила,Хорошая футболка,2.0,M,2


In [24]:
feedback_m.to_csv('FInal_dataset_M.csv')

## Если нужно лемматизировать

In [41]:
stops = set(stopwords.words('russian') + ["это", "весь","всё", "футболка", "очень"]) # определили список стоп-слов
morph = MorphAnalyzer()

In [42]:
def normalize(text):
    tokens = re.findall('[а-яёa-z0-9]+', str(text).lower())
    normalized_text = [morph.parse(word)[0].normal_form for word \
                                                            in tokens]
    normalized_text = [word for word in normalized_text if len(word) > 2 and word not in stops]
    
    return normalized_text

In [43]:
feedback_normalized = [' '.join(normalize(elem)) for elem in feedback]
feedback_m = feedback_m.assign(feedback=feedback_normalized)
feedback_m# нормализовали текст отзыва

ValueError: Length of values (1096) does not match length of index (1695)

In [ ]:
reviews_f.to_csv("Final_dataset_M_norm.csv")

## Открываем датасет Ж

In [26]:
feedback_f = pd.read_csv('Финальный датасет Ж.csv')

In [27]:
Final_dataset = feedback_m.append(feedback_f)

In [28]:
Final_dataset

,stars,name,feedback,avg_sent,gender,num_words,Unnamed: 0
0,100,Стас,Взял,1.000000,M,1,NaN
1,80,Рахман,Неплохая футболка еще не успел долго проносить,7.000000,M,7,NaN
2,100,Андрю,Норм,1.000000,M,1,NaN
3,100,Сергей,👍🏼,1.000000,M,1,NaN
4,100,Алексей,Нормальная футболка По дому ходить пойдёт дл...,5.000000,M,10,NaN
...,...,...,...,...,...,...,...
800,100,Юлия,👍🏻,1.000000,F,1,800.0
801,100,Татьяна,Отличная футболка для сна Размер в размер,3.500000,F,7,801.0
802,20,Захарова Ира,Махровая футболка есть такие кто захочет тако...,4.666667,F,14,802.0
803,60,надежда,футболка как махровое полотенце отказалась,5.000000,F,5,803.0


In [29]:
Final_dataset = Final_dataset.drop(['Unnamed: 0'], axis=1, inplace=False) #Как отбросить первую колонку?

In [30]:
Final_dataset

,stars,name,feedback,avg_sent,gender,num_words
0,100,Стас,Взял,1.000000,M,1
1,80,Рахман,Неплохая футболка еще не успел долго проносить,7.000000,M,7
2,100,Андрю,Норм,1.000000,M,1
3,100,Сергей,👍🏼,1.000000,M,1
4,100,Алексей,Нормальная футболка По дому ходить пойдёт дл...,5.000000,M,10
...,...,...,...,...,...,...
800,100,Юлия,👍🏻,1.000000,F,1
801,100,Татьяна,Отличная футболка для сна Размер в размер,3.500000,F,7
802,20,Захарова Ира,Махровая футболка есть такие кто захочет тако...,4.666667,F,14
803,60,надежда,футболка как махровое полотенце отказалась,5.000000,F,5


In [31]:
Final_dataset ['stars'] = Final_dataset['stars'].replace(20, 1)
Final_dataset ['stars'] = Final_dataset['stars'].replace(40, 2)
Final_dataset ['stars'] = Final_dataset['stars'].replace(60, 3)
Final_dataset ['stars'] = Final_dataset['stars'].replace(80, 4)
Final_dataset ['stars'] = Final_dataset['stars'].replace(100, 5)

In [32]:
Final_dataset

,stars,name,feedback,avg_sent,gender,num_words
0,5,Стас,Взял,1.000000,M,1
1,4,Рахман,Неплохая футболка еще не успел долго проносить,7.000000,M,7
2,5,Андрю,Норм,1.000000,M,1
3,5,Сергей,👍🏼,1.000000,M,1
4,5,Алексей,Нормальная футболка По дому ходить пойдёт дл...,5.000000,M,10
...,...,...,...,...,...,...
800,5,Юлия,👍🏻,1.000000,F,1
801,5,Татьяна,Отличная футболка для сна Размер в размер,3.500000,F,7
802,1,Захарова Ира,Махровая футболка есть такие кто захочет тако...,4.666667,F,14
803,3,надежда,футболка как махровое полотенце отказалась,5.000000,F,5


In [34]:
Final_dataset.to_csv('Финальный датасет.csv')

In [6]:
Final_dataset = pd.read_csv('Финальный датасет.csv')

In [12]:
Final_dataset

,Unnamed: 0,stars,name,feedback,avg_sent,gender,num_words
0,0,5,Стас,Взял,1.000000,M,1
1,1,4,Рахман,Неплохая футболка еще не успел долго проносить,7.000000,M,7
2,2,5,Андрю,Норм,1.000000,M,1
3,3,5,Сергей,👍🏼,1.000000,M,1
4,4,5,Алексей,Нормальная футболка По дому ходить пойдёт дл...,5.000000,M,10
...,...,...,...,...,...,...,...
2495,800,5,Юлия,👍🏻,1.000000,F,1
2496,801,5,Татьяна,Отличная футболка для сна Размер в размер,3.500000,F,7
2497,802,1,Захарова Ира,Махровая футболка есть такие кто захочет тако...,4.666667,F,14
2498,803,3,надежда,футболка как махровое полотенце отказалась,5.000000,F,5


In [16]:
!pip install emojis
import emojis

You should consider upgrading via the 'C:\Users\ДНС\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [27]:
def emoji(x):
    try:
        x = emojis.count(x)
    except TypeError: 
        x = re.sub("[^a-zA-Z]",  # Search for all non-letters
                          " ",          # Replace all non-letters with spaces
                          str(x))
        x = emojis.count(x)
    return x

In [29]:
Final_dataset['emoji'] = Final_dataset['feedback'].apply(lambda x: emoji(x))
Final_dataset

,Unnamed: 0,stars,name,feedback,avg_sent,gender,num_words,emoji
0,0,5,Стас,Взял,1.000000,M,1,0
1,1,4,Рахман,Неплохая футболка еще не успел долго проносить,7.000000,M,7,0
2,2,5,Андрю,Норм,1.000000,M,1,0
3,3,5,Сергей,👍🏼,1.000000,M,1,1
4,4,5,Алексей,Нормальная футболка По дому ходить пойдёт дл...,5.000000,M,10,0
...,...,...,...,...,...,...,...,...
2495,800,5,Юлия,👍🏻,1.000000,F,1,1
2496,801,5,Татьяна,Отличная футболка для сна Размер в размер,3.500000,F,7,0
2497,802,1,Захарова Ира,Махровая футболка есть такие кто захочет тако...,4.666667,F,14,0
2498,803,3,надежда,футболка как махровое полотенце отказалась,5.000000,F,5,0


In [31]:
Final_dataset = Final_dataset.drop(['Unnamed: 0'], axis=1, inplace=False)

In [32]:
Final_dataset.to_csv('Финальный датасет.csv')